# import def func

In [3]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [4]:
f'lastpTDX_DF{resample}' in locals()

NameError: name 'resample' is not defined

In [5]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20):
    col_src = [
        "name", "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
#     tdx_df = f'lastpTDX_DF_{resample}'
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [6]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [7]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [8]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-19',
 code
 000533    顺钠股份
 Name: name, dtype: object)

In [9]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/20/25 22:41:41'

code
300340   2025-06-20 15:00:00
003019   2025-06-20 15:00:00
Name: ticktime, dtype: datetime64[ns]

461.68760435183844

#### 实时成交

In [10]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(2767032, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-20 14:58:58,66.9,67.88,66.75,67.55,16223684,67.11
2025-06-20 14:59:40,66.9,67.88,66.75,67.55,16223684,67.18
2025-06-20 15:00:01,66.9,67.88,66.75,67.55,16398684,67.07


1.0

总量:8715.97 亿


ticktime    2025-06-21 00:00:00
close                  67.21773
volume                     0.13
vol                        16.4
amount                    11.02
Name: 603019, dtype: object

'2025-06-20'

#### 读取实时成交个股

In [11]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5420, 5),
         volume   name
 code                 
 300308    0.73   中际旭创
 300468    0.71   四方精创
 600519    0.57   贵州茅台
 300476    0.55   胜宏科技
 300502    0.54    新易盛
 601318    0.53   中国平安
 300773    0.49    拉卡拉
 002475    0.45   立讯精密
 002463    0.45   沪电股份
 002104    0.43   恒宝股份
 000858    0.41  五 粮 液
 300059    0.39   东方财富
 300750    0.38   宁德时代
 003040    0.36    楚天龙
 002384    0.35   东山精密
 002017    0.33   东信和平
 603300    0.32   海南华铁
 300255    0.31   常山药业
 002050    0.31   三花智控
 002594    0.29    比亚迪)

##### 合并实时成交量及SinaDM

In [12]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey

In [193]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d',
             '新高上upper':'lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper',
            }   
filter_keys=list(filter_query.keys())

In [194]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [195]:
filterIDX=3
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [196]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx 

[2025-06-21 15:09:57,556] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:3 code:['600705', '601028']


:0 b1>:5184 it:0.5 lastpTDX_DF_3d in locals
tdxdataT:2.46
DLTDXE:2.47 

In [197]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
t3d_code = h5_3d.query(f'{get_filter(filterIDX)}')
t3d_code = h5_3d.query('lastp1d > lastp2d and lastp2d > lastp3d')
show_tdx_data(t3d_code,limit=10)

新高上upper: lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper
Count:695 221


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
688375,国博电子,0,2025-04-08,57.66,60.12,44.81,0.1,57.41,44.81,0.1,0.1
601811,新华文轩,0,2025-05-14,14.57,15.45,14.09,0.0,14.68,14.62,-0.1,0.1
605318,法狮龙,0,2025-06-13,28.06,31.23,24.91,1.3,28.50,25.72,0.1,1.3
688100,威胜信息,0,2025-06-13,33.41,38.56,30.78,1.3,33.31,31.52,0.0,1.2
688090,瑞松科技,0,2025-04-08,40.09,45.88,27.05,0.1,39.94,29.09,-1.1,-1.1
301252,同星科技,0,2025-04-08,22.98,24.73,16.58,0.1,23.20,18.09,-1.1,0.1
688570,天玛智控,0,2025-04-08,19.45,20.50,16.31,1.3,19.67,16.31,0.1,0.2
301045,天禄科技,0,2025-04-08,21.88,21.80,16.50,1.3,22.02,17.20,1.2,1.3
605218,伟时电子,0,2025-04-08,21.25,22.62,15.53,0.2,21.39,17.09,-1.1,1.3


In [198]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# t3d_code = h5_3d.query(f'{get_filter(1)}')
# show_tdx_data(t3d_code)

### resample W

In [199]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5184 it:0.4 lastpTDX_DF_w in locals


In [200]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
# tw_code = h5_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query('lastp1d > ma51d and lastp2d > ma51d')
show_tdx_data(tw_code,limit=10)

Count:1238 223


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
601028,玉龙股份,0,2025-02-28,12.83,14.39,10.90,0.0,0.00,11.14,1.30,-1.1
839946,华阳变速,0,2025-01-03,9.42,10.67,5.98,-0.9,9.58,6.17,0.00,-1.1
001238,浙江正特,0,2024-12-27,37.06,36.31,24.91,1.2,37.19,25.73,33.41,-0.9
605296,神农集团,0,2025-02-07,32.31,30.27,24.55,0.1,32.69,24.55,1.30,-1.1
688321,微芯生物,0,2025-04-18,21.17,21.30,14.86,-1.1,21.17,16.00,36.00,0.0
603713,密尔克卫,0,2025-04-11,52.12,55.44,42.50,-1.1,52.28,42.50,19.41,0.0
603737,三棵树,0,2025-01-10,34.69,37.09,26.20,0.0,34.84,26.94,0.10,-1.1
301390,经纬股份,0,2025-01-10,34.22,39.41,28.41,0.0,34.34,29.30,20.30,-1.1
301052,果麦文化,0,2025-01-10,39.00,36.28,22.06,-0.9,39.06,22.23,1.10,1.2


### resample M

In [201]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx :0 b1>:5184 it:0.5 lastpTDX_DF_m in locals


In [202]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
tm_code = h5_m.query('lastl1d > lastl2d and lastl2d > lastl3d and lastp1d > ma51d')
# tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code)

Count:2304 221


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
601028,玉龙股份,0,2025-02-28,12.83,14.58,10.52,1.10,0.00,11.14,1.1,0.00
839946,华阳变速,0,2025-01-27,9.42,10.46,2.77,1.10,9.58,6.18,0.0,48.50
001238,浙江正特,0,2024-09-30,37.06,32.69,19.03,1.20,37.19,20.59,0.0,16.30
003002,壶化股份,0,2024-09-30,25.80,23.50,9.36,-0.90,25.98,10.07,0.0,1.20
688321,微芯生物,0,2025-01-27,21.17,23.00,15.15,1.30,21.17,16.52,-1.1,15.30
603713,密尔克卫,0,2025-04-30,52.12,70.50,40.56,1.30,52.28,42.50,-1.0,-0.80
688375,国博电子,0,2025-01-27,57.66,53.97,31.62,0.10,57.41,43.16,-1.1,33.41
300779,惠城环保,0,2024-09-30,163.50,144.00,35.19,1.20,165.38,44.09,-1.0,48.00
301390,经纬股份,0,2025-01-27,34.22,34.56,24.66,1.20,34.34,28.41,-1.1,1.20


### resample D

In [214]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d')
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)

initdx 

[2025-06-21 15:13:57,772] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:3 code:['600705', '601028']


:0 b1>:5184 it:0.5 lastpTDX_DF_d in locals
tdxdataT:2.6
DLTDXE:2.60 

In [215]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
td_code = h5_d.query(f'{get_filter(filterIDX)}')
td_code = h5_d.query(f'lasth1d > lasth2d and close > lastp1d')

show_tdx_data(td_code,limit=10)
if len(td_code) < 30:
    write_code_to_blk(td_code.index,'065',rewrite=True)

新高上upper: lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper
Count:487 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,0,2025-04-07,52.53,43.47,18.39,1.20,48.81,13.18,0.30,-1.0
301038,深水规院,15,2025-04-07,17.27,15.36,13.51,0.20,16.09,11.53,-0.10,-0.9
301292,海科新源,13,2025-04-07,23.52,15.62,14.50,48.09,23.52,11.46,28.09,-0.1
300429,强力新材,155,2025-04-28,14.27,12.72,10.92,1.20,14.23,10.44,0.20,-1.1
300480,光力科技,0,2025-04-07,15.59,13.70,12.66,1.20,15.09,10.98,0.10,-1.1
300340,科恒股份,92,2025-04-07,20.64,18.86,9.68,0.10,19.86,7.90,25.30,-1.1
300589,江龙船艇,36,2025-04-07,15.12,12.08,11.12,31.80,14.25,10.10,1.30,1.3
300410,正业科技,71,2025-04-07,7.45,6.21,5.51,1.20,7.43,4.12,1.20,-1.1
301150,中一科技,15,2025-04-07,21.91,17.03,15.16,37.31,20.45,12.35,1.30,-1.1


#### filter test

In [216]:
# h5_t_code=h5_w.query('lasth1d > upper1 and lasth1d >lasth2d   and high >boll and percent > maxp')
h5_t_code=h5_d.query('lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d and lastp2d < upper2 and lasth1d >upper')
show_tdx_data(h5_t_code,limit=5)

Count:117 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
301150,中一科技,15,2025-04-07,21.91,17.03,15.16,37.31,20.45,12.35,1.3,-1.1
003040,楚天龙,84,2025-04-08,23.33,19.20,12.37,47.81,21.95,10.71,0.3,-1.1
300695,兆丰股份,0,2025-04-08,62.88,69.19,51.22,1.30,61.94,34.88,-0.1,-1.0
603256,宏和科技,36,2025-04-08,13.68,12.58,9.35,1.20,12.77,7.01,1.2,-1.1
831152,昆工科技,0,2025-04-07,24.44,21.70,18.80,43.09,24.00,15.00,0.1,0.0


In [217]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
show_tdx_data(td_code2,limit=10)


Count:41 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,0,2025-04-07,52.53,43.47,18.39,1.20,48.81,13.18,0.3,-1.0
839719,宁新新材,0,2025-04-07,23.70,17.45,11.23,1.20,22.64,8.90,0.0,1.2
300269,联建光电,218,2025-04-07,6.17,4.18,3.83,52.09,6.01,2.95,1.2,-1.1
003040,楚天龙,84,2025-04-08,23.33,19.20,12.37,47.81,21.95,10.71,0.3,-1.1
600110,诺德股份,19,2025-04-29,5.15,3.75,3.23,44.91,5.15,2.89,1.3,-1.1
601606,长城军工,134,2025-04-07,18.36,14.18,12.61,45.59,18.12,10.79,1.3,1.3
600113,浙江东日,13,2025-04-07,20.69,14.50,13.68,45.19,19.38,12.63,17.3,1.2
002235,安妮股份,269,2025-04-08,9.03,7.12,5.17,1.30,8.88,4.33,0.0,17.3
603256,宏和科技,36,2025-04-08,13.68,12.58,9.35,1.20,12.77,7.01,1.2,-1.1


#### 多周期筛选

In [228]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(td_code.index))
code_f =td_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()
if len(code_f) > 0:
    show_tdx_data(td_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:2304
tw_code:1238
t3d_code:695
td_code:487
Count:44 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,0,2025-04-07,52.53,43.47,18.39,1.20,48.81,13.18,0.3,-1.0
300340,科恒股份,92,2025-04-07,20.64,18.86,9.68,0.10,19.86,7.90,25.3,-1.1
300410,正业科技,71,2025-04-07,7.45,6.21,5.51,1.20,7.43,4.12,1.2,-1.1
839719,宁新新材,0,2025-04-07,23.70,17.45,11.23,1.20,22.64,8.90,0.0,1.2
003040,楚天龙,84,2025-04-08,23.33,19.20,12.37,47.81,21.95,10.71,0.3,-1.1
002235,安妮股份,269,2025-04-08,9.03,7.12,5.17,1.30,8.88,4.33,0.0,17.3
002940,昂利康,4,2025-04-08,31.94,28.98,13.25,20.30,31.53,12.12,1.3,-1.0
603151,邦基科技,15,2025-04-07,21.34,17.12,12.88,-1.10,21.25,11.64,17.3,1.3
300046,台基股份,0,2025-04-07,33.59,32.44,29.38,1.20,35.19,23.91,-1.0,0.1


#### check data

In [229]:
'300998' in code_f
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

True

44

In [230]:
show_tdx_data(td_code.loc[code_f])

Count:44 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,0,2025-04-07,52.53,43.47,18.39,1.20,48.81,13.18,0.30,-1.0
300340,科恒股份,92,2025-04-07,20.64,18.86,9.68,0.10,19.86,7.90,25.30,-1.1
300410,正业科技,71,2025-04-07,7.45,6.21,5.51,1.20,7.43,4.12,1.20,-1.1
839719,宁新新材,0,2025-04-07,23.70,17.45,11.23,1.20,22.64,8.90,0.00,1.2
003040,楚天龙,84,2025-04-08,23.33,19.20,12.37,47.81,21.95,10.71,0.30,-1.1
002235,安妮股份,269,2025-04-08,9.03,7.12,5.17,1.30,8.88,4.33,0.00,17.3
002940,昂利康,4,2025-04-08,31.94,28.98,13.25,20.30,31.53,12.12,1.30,-1.0
603151,邦基科技,15,2025-04-07,21.34,17.12,12.88,-1.10,21.25,11.64,17.30,1.3
300046,台基股份,0,2025-04-07,33.59,32.44,29.38,1.20,35.19,23.91,-1.00,0.1


### Write toblk

In [233]:
# write_code_to_blk(code_f,'077',rewrite=False)

write blk [Y] or [N]:y
Rewrite code [Y] or append [N](defalut:N):y
all write to /Users/Johnson/Documents/Johnson/WinTools/new_tdx2/T0002/blocknew/077.blk:44
all write to /Users/Johnson/Documents/Johnson/WinTools/new_tdx/T0002/blocknew/077.blk:44
write:44 block_path:/Users/Johnson/Documents/Johnson/WinTools/new_tdx2/T0002/blocknew/077.blk
